In [1]:
from yahoo_fantasy_api import League, Team
from yahoo_oauth import OAuth2

In [2]:
oauth = OAuth2(None, None, from_file='../auth/oauth2yahoo.json')
league = League(oauth, league_id='461.l.6288') #game_key.l.leauge_key


[2025-11-20 14:50:47,937 DEBUG] [yahoo_oauth.oauth.__init__] Checking 
[2025-11-20 14:50:47,940 DEBUG] [yahoo_oauth.oauth.token_is_valid] ELAPSED TIME : 2295.6471195220947
[2025-11-20 14:50:47,941 DEBUG] [yahoo_oauth.oauth.token_is_valid] TOKEN IS STILL VALID


In [3]:
my_team_code = "461.l.6288.t.7" #game_key.l.league_key.t.team_key
team = Team(oauth, team_key=my_team_code)
team.details()

{'team_key': '461.l.6288.t.7',
 'team_id': '7',
 'name': 'Milwaukee Millies',
 'is_owned_by_current_login': 1,
 'url': 'https://football.fantasysports.yahoo.com/f1/6288/7',
 'team_logos': [{'team_logo': {'size': 'large',
    'url': 'https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/06430359b11625d0c85f4b3c0c9215fe7703b0ec071b49d2f932aa1c76b74cb3.jpg'}}],
 'previous_season_team_rank': 1,
 'division_id': '3',
 'number_of_moves': 10,
 'number_of_trades': 0,
 'roster_adds': {'coverage_type': 'week', 'coverage_value': 12, 'value': '0'},
 'league_scoring_type': 'head',
 'has_draft_grade': 1,
 'draft_grade': 'B',
 'draft_recap_url': 'https://football.fantasysports.yahoo.com/f1/6288/7/draftrecap',
 'managers': [{'manager': {'manager_id': '7',
    'nickname': 'Aidan',
    'guid': '4KAKNEK4VYYTC7DRQGST2OWGLE',
    'is_current_login': '1',
    'image_url': 'https://s.yimg.com/ag/images/default_user_profile_pic_64sq.jpg',
    'felo_score': '721',
    'felo_tier': 'g

In [4]:
import pandas as pd

In [5]:
roster = team.roster(1) #not sure why this needs a 1, works for all without it except team 4
players = pd.json_normalize(roster)
print(players)

    player_id                name status position_type eligible_positions  \
0       29235          Jared Goff                    O               [QB]   
1       33393       Ja'Marr Chase   SUSP             O   [WR, W/R, W/R/T]   
2       30994            DJ Moore                    O   [WR, W/R, W/R/T]   
3       30972      Saquon Barkley                    O   [RB, W/R, W/R/T]   
4       30218        James Conner     IR             O   [RB, W/R, W/R/T]   
5       34085       Jake Ferguson                    O        [TE, W/R/T]   
6       31005          Nick Chubb                    O   [RB, W/R, W/R/T]   
7       40063         Jayden Reed     IR             O   [WR, W/R, W/R/T]   
8       31017      Christian Kirk                    O   [WR, W/R, W/R/T]   
9       40042      Jordan Addison                    O   [WR, W/R, W/R/T]   
10      40900      Caleb Williams                    O               [QB]   
11      40119        Tyjae Spears                    O   [RB, W/R, W/R/T]   

In [6]:
#get player id's to use for data querying
player_id_list = players['player_id'].tolist()
player_stats_2025 = league.player_stats(player_id_list, req_type='season', season=2025)

In [7]:
#create a dataframe of the normalized stats
df = pd.json_normalize(player_stats_2025)
df.fillna(0, inplace=True)
df['total_points'] = pd.to_numeric(df['total_points']) #values were str causing issues with sorting
# print(df)

In [8]:
#table is large, check just columns that we have in table to see what is important
print(df.columns)

Index(['player_id', 'name', 'position_type', 'Pass Yds', 'Pass TD', 'Int',
       'Rush Att', 'Rush Yds', 'Rush TD', 'Targets', 'Rec', 'Rec Yds',
       'Rec TD', 'Ret Yds', 'Ret TD', '2-PT', 'Fum Lost', 'Fum Ret TD',
       'total_points', 'FG 0-19', 'FG 20-29', 'FG 30-39', 'FG 40-49', 'FG 50+',
       'PAT Made', 'Pts Allow', 'Sack', 'Fum Rec', 'TD', 'Safe', 'Blk Kick',
       'TFL', 'Def Yds Allow', '3 and Outs', 'Pts Allow 0', 'Pts Allow 1-6',
       'Pts Allow 7-13', 'Pts Allow 14-20', 'Pts Allow 21-27',
       'Pts Allow 28-34', 'Pts Allow 35+', 'Yds Allow Neg', 'Yds Allow 0-99',
       'Yds Allow 100-199', 'Yds Allow 200-299', 'Yds Allow 300-399',
       'Yds Allow 400-499', 'Yds Allow 500+', 'XPR', 'Tack Solo', 'Tack Ast',
       'Fum Force', 'Pass Def', 'TO Ret Yds'],
      dtype='object')


In [9]:
player_by_total_points = df[['name','total_points']] 
position = players['eligible_positions']
player_by_total_points = player_by_total_points.join(position)
player_by_total_points = player_by_total_points.sort_values('total_points',ascending=False)
print(player_by_total_points)


                  name  total_points eligible_positions
18         Los Angeles        280.30              [DEF]
17         Kansas City        254.60              [DEF]
10      Caleb Williams        218.56               [QB]
0           Jared Goff        215.80               [QB]
1        Ja'Marr Chase        198.70   [WR, W/R, W/R/T]
3       Saquon Barkley        151.90   [RB, W/R, W/R/T]
5        Jake Ferguson        141.00        [TE, W/R/T]
12       Juwan Johnson        108.10        [TE, W/R/T]
2             DJ Moore         95.88   [WR, W/R, W/R/T]
13      Bhayshul Tuten         94.30   [RB, W/R, W/R/T]
9       Jordan Addison         89.80   [WR, W/R, W/R/T]
16      Brandon Aubrey         86.00                [K]
21           Zack Baun         77.50               [LB]
24     Ernest Jones IV         74.40               [LB]
6           Nick Chubb         71.80   [RB, W/R, W/R/T]
19  Minkah Fitzpatrick         67.20               [DB]
20         Maxx Crosby         63.90            

In [10]:
goff = pd.json_normalize(league.player_details('Jared Goff'))
print(goff.columns)

Index(['player_key', 'player_id', 'url', 'editorial_player_key',
       'editorial_team_key', 'editorial_team_full_name', 'editorial_team_abbr',
       'editorial_team_url', 'uniform_number', 'display_position', 'image_url',
       'is_undroppable', 'position_type', 'primary_position',
       'eligible_positions', 'eligible_positions_to_add', 'has_player_notes',
       'player_notes_last_timestamp', 'name.full', 'name.first', 'name.last',
       'name.ascii_first', 'name.ascii_last', 'bye_weeks.week',
       'is_keeper.status', 'is_keeper.cost', 'is_keeper.kept', 'headshot.url',
       'headshot.size', 'player_stats.0.coverage_type',
       'player_stats.0.season', 'player_stats.stats',
       'player_advanced_stats.0.coverage_type',
       'player_advanced_stats.0.season', 'player_advanced_stats.stats',
       'player_points.0.coverage_type', 'player_points.0.season',
       'player_points.total'],
      dtype='object')


In [11]:
print(goff['bye_weeks.week'])

0    8
Name: bye_weeks.week, dtype: object


In [12]:
#graphing
import matplotlib.pyplot as plt

In [13]:
print(player_by_total_points['name'])

18           Los Angeles
17           Kansas City
10        Caleb Williams
0             Jared Goff
1          Ja'Marr Chase
3         Saquon Barkley
5          Jake Ferguson
12         Juwan Johnson
2               DJ Moore
13        Bhayshul Tuten
9         Jordan Addison
16        Brandon Aubrey
21             Zack Baun
24       Ernest Jones IV
6             Nick Chubb
19    Minkah Fitzpatrick
20           Maxx Crosby
15          Jake Elliott
11          Tyjae Spears
23           Tyler Nubin
14     Ja'Tavion Sanders
22      Quinnen Williams
4           James Conner
8         Christian Kirk
7            Jayden Reed
Name: name, dtype: object


In [14]:
# plt.figure(figsize=(30, 25))
# plt.bar(player_by_total_points['name'], player_by_total_points['total_points'])

In [15]:
#testing functions

In [16]:
goff_details = pd.json_normalize(league.player_details("Jared Goff"))
goff_player_id = goff_details['player_id'].tolist()
print(goff_player_id)

['29235']


In [17]:
goff_stats = league.player_stats(goff_player_id, req_type='season', season=2025)

In [18]:
goff_stats = pd.json_normalize(goff_stats)
print(goff_stats)

   player_id        name position_type  Pass Yds  Pass TD  Int  Rush Att  \
0      29235  Jared Goff             O    2490.0     21.0  4.0      12.0   

   Rush Yds  Rush TD  Targets  Rec  Rec Yds  Rec TD  Ret Yds  Ret TD  2-PT  \
0      22.0      0.0      0.0  0.0      0.0     0.0      0.0     0.0   0.0   

   Fum Lost  Fum Ret TD total_points  
0       2.0         0.0       215.80  


In [19]:
print(goff_stats._get_value(0, 'Pass TD'))

21.0


In [20]:
#find total passing TDs for a specific player name
player_name = 'Caleb Williams'
player_details = pd.json_normalize(league.player_details(player_name))
player_id = player_details['player_id'].tolist()
player_stats = league.player_stats(player_id, req_type='season', season=2025)
player_stats = pd.json_normalize(player_stats)
# print(player_stats._get_value(0, 'Pass TD'))
print(player_stats)

   player_id            name position_type  Pass Yds  Pass TD  Int  Rush Att  \
0      40900  Caleb Williams             O    2329.0     13.0  4.0      52.0   

   Rush Yds  Rush TD  Targets  Rec  Rec Yds  Rec TD  Ret Yds  Ret TD  2-PT  \
0     272.0      3.0      2.0  2.0     22.0     1.0      0.0     0.0   0.0   

   Fum Lost  Fum Ret TD total_points  
0       0.0         0.0       218.56  


In [21]:
import sys
sys.path.append('../')

In [22]:
from src.data_collection import *

In [23]:
print(get_player_stats(league, player_name))

   player_id            name position_type  Pass Yds  Pass TD  Int  Rush Att  \
0      40900  Caleb Williams             O    2329.0     13.0  4.0      52.0   

   Rush Yds  Rush TD  Targets  Rec  Rec Yds  Rec TD  Ret Yds  Ret TD  2-PT  \
0     272.0      3.0      2.0  2.0     22.0     1.0      0.0     0.0   0.0   

   Fum Lost  Fum Ret TD total_points  
0       0.0         0.0       218.56  
